# 🏭 Industria y Manufactura: OEE, calidad y throughput
KPIs: OEE, scrap, MTBF/MTTR, lead time, first pass yield.

## Casos de uso
- Control estadístico de procesos (SPC)
- Mantenimiento predictivo y calidad visual
- Gemelos digitales y simulación de cuellos de botella
- Trazabilidad de lote y recall

In [ ]:
# Cálculo simple de OEE
def oee(disponibilidad, rendimiento, calidad):
    return disponibilidad * rendimiento * calidad
round(oee(0.9, 0.92, 0.98), 3)

## Arquitectura
- Ingesta PLC/SCADA/IoT, series de tiempo
- Modelos de visión y features en borde (edge)
- Integración MES/ERP y tableros de planta

## 💼 Valor en Manufactura
- OEE ↑ (disponibilidad, rendimiento, calidad) → mayor throughput.
- Scrap ↓ con control estadístico de procesos y visión.
- Trazabilidad de lote y recall eficiente.


In [ ]:
# Práctica: OEE por turno (toy)
import pandas as pd
turnos = pd.DataFrame({
    'turno':['A','B','C'],
    'disp':[0.92,0.88,0.9],
    'rend':[0.93,0.9,0.92],
    'cal':[0.98,0.95,0.97],
})
turnos['OEE'] = turnos['disp']*turnos['rend']*turnos['cal']
print(turnos)
print('OEE promedio:', round(turnos['OEE'].mean(),3))


## 🏗️ Arquitectura MES/ERP/SCADA
- Ingesta PLC/SCADA/IoT; normalización de señales y series de tiempo.
- Integración MES/ERP; features para visión y mantenimiento.
- Tableros de línea/planta y alertas.


In [ ]:
# Visual: OEE por turno
import plotly.express as px
fig = px.bar(turnos, x='turno', y='OEE', title='OEE por turno')
fig.show()


## ✅ Checklist de estrategia de negocio y datos
- Objetivo (OEE↑, scrap↓, throughput↑): …
- KPIs: OEE, FPY, MTBF/MTTR, lead time.
- Palancas: SPC, visión, mantenimiento, trazabilidad.
- Datos/calidad: PLC/SCADA/IoT, calibración, tests.
- Gobierno: MES/ERP, dominios, seguridad.
- SLOs: latencia tableros, costo, disponibilidad.
- ROI/supuestos: …
- Riesgos: variabilidad, calidad sensor, seguridad.


## 🔗 Puente Estrategia ↔ Ingeniería de Datos
- OKR: OEE ≥ 0.90 y scrap -15% → KPI negocio: OEE, scrap → KPI datos: disponibilidad señales, calidad visión → Capacidades: SCADA/PLC + visión + control SPC → Decisión: ajustes de línea/mantenimiento → Impacto: throughput↑, scrap↓.

```
[PLC/SCADA]→[Series tiempo]→[Features calidad]→[OEE/Scrap]→[Umbrales]→[Ajuste de línea]→[Δ margen]
```


In [ ]:
# contrato_de_datos (manufactura)
contrato_datos = {
    "objetivo_negocio": "OEE ≥ 0.90 y scrap -15%",
    "kpi_negocio": ["OEE", "Scrap %"],
    "kpi_datos": {"uptime_senales_%": ">=99%", "deteccion_defectos_prec": ">=95%"},
    "dataset": ["series_tiempo_maquinas", "vision_defectos"],
    "propietario": "Dominio Operaciones Planta",
    "slo": {"latencia_alerta_s": "<15"},
    "decision": "Ajuste de parámetros y mantenimiento",
    "impacto_economico_usd": 210000,
}
print(contrato_datos)


## 📋 Caso de uso: Reducción de scrap con visión y SPC

**Contexto de negocio:** Una planta de manufactura tiene 8% de scrap (producto defectuoso). Cada punto de scrap = $1.2M/año en pérdidas. Objetivo: scrap <3%.

**Problema sin ingeniería de datos:**
- Inspección manual al final de línea (detecta defectos cuando ya se produjeron 1000 unidades).
- Parámetros de máquina sin monitoreo en tiempo real.
- Análisis de causas raíz manual (toma semanas).

**Solución con ingeniería de datos:**
1. Ingesta PLC/SCADA: temperatura, presión, velocidad de cada estación → TSDB.
2. Visión computacional: inspección automática en línea, detección de defectos con 98% precisión.
3. Control estadístico de procesos (SPC): límites de control dinámicos, alertas pre-defecto.
4. Modelo de datos: serie temporal + eventos de defectos → análisis de correlación automático.
5. Dashboard de línea: operadores ven en tiempo real cuándo un parámetro sale de control.

**Resultado:**
- Scrap: 8% → 2.8%.
- Detección de defectos: fin de línea → en-proceso (ahorro de 90% de material).
- OEE: 0.78 → 0.88 (menos paradas por calidad).
- Ahorro anual: $6.2M + mejora de margen por menos retrabajo.


In [ ]:
# Ejercicio: límites de control SPC (Statistical Process Control)
import pandas as pd
import numpy as np

# Simular mediciones de temperatura en línea de producción
np.random.seed(1)
mediciones = pd.DataFrame({
    'hora': range(1, 51),
    'temperatura': 180 + np.random.randn(50) * 2,
})

# Calcular límites de control (3 sigma)
media = mediciones['temperatura'].mean()
sigma = mediciones['temperatura'].std()
mediciones['LCS'] = media + 3*sigma  # Límite de Control Superior
mediciones['LCI'] = media - 3*sigma  # Límite de Control Inferior
mediciones['fuera_control'] = (
    (mediciones['temperatura'] > mediciones['LCS']) |
    (mediciones['temperatura'] < mediciones['LCI'])
)

fuera = mediciones[mediciones['fuera_control'] == True]
print(f"Media: {media:.1f}°C, Sigma: {sigma:.2f}")
print(f"Límites de control: [{media-3*sigma:.1f}, {media+3*sigma:.1f}]")
print(f"\nPuntos fuera de control: {len(fuera)}")
print(fuera[['hora', 'temperatura', 'LCS', 'LCI']].head())
print("\n💡 Pipeline genera alerta cuando se detecta tendencia antes de defecto")
